In [86]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import display

In [87]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import  LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

In [88]:
df = pd.read_csv('sales_train.csv')
items = pd.read_csv('items.csv')
item_cats = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

In [89]:
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [90]:
df.describe(exclude = None)

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [91]:
df.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [92]:
df.shape

(2935849, 6)

In [93]:
df['date'] = pd.to_datetime(df.date, format = '%d.%m.%Y')

In [94]:
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [95]:
df1 = df.copy()
df1.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [96]:
df1['upload_month'] = df1.date.dt.month
del df1['date']
df1.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,upload_month
0,0,59,22154,999.00,1.0,1
1,0,25,2552,899.00,1.0,1
2,0,25,2552,899.00,-1.0,1
3,0,25,2554,1709.05,1.0,1
4,0,25,2555,1099.00,1.0,1


In [97]:
X = df1.drop('item_cnt_day', axis = 1)
y = df1['item_cnt_day']
display(X.head())
display(y.head())

,date_block_num,shop_id,item_id,item_price,upload_month
0,0,59,22154,999.00,1
1,0,25,2552,899.00,1
2,0,25,2552,899.00,1
3,0,25,2554,1709.05,1
4,0,25,2555,1099.00,1


0    1.0
1    1.0
2   -1.0
3    1.0
4    1.0
Name: item_cnt_day, dtype: float64

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 1)

In [99]:
reg_methods = [LinearRegression(), RandomForestRegressor(random_state = 1)]
names_reg = ['Linear Regression','Random Forest']

In [100]:
def get_fits(reg_methods):
    fits = []
    for method in reg_methods:
        random.seed(1)
        reg = method
        reg.fit(X_train,y_train)
        fits.append(reg)
    return fits

fits = get_fits(reg_methods)

In [101]:
def get_scores(fits, names_reg, X_data, y_data):
    reg_reports = []
    for reg in fits:
        y_pred = reg.predict(X_data)
        mse = mean_squared_error(y_data, y_pred)
        reg_reports.append(mse)
        
    index = names_reg
    scores = pd.DataFrame(reg_reports, index = index, columns = ['MSE'])
    return scores
    
scores = get_scores(fits,names_reg, X_test, y_test)
scores

,MSE
Linear Regression,12.699874
Random Forest,11.430137


In [102]:
#Note: try to analyze seasonality, when sales were higher, etc.